In [46]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Activation, dot
import keras.backend as K
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import uncurl
from uncurl import max_variance_genes, run_state_estimation
import umap
from tsne import bh_sne

In [6]:
mat = scipy.io.loadmat('../data/10x_pooled_400.mat')

In [7]:
dat = np.log10(1 + mat['data'].toarray().astype(np.float32))
genes = uncurl.max_variance_genes(dat, nbins=5, frac=0.2)
dat = dat[genes,:]
dat_T = dat.transpose()
(c,g) = dat_T.shape
print dat_T.shape

(400, 2747)


In [98]:
def OptimizeW(M,dat_T,k):
    (c,g) = dat_T.shape

    enc_dim1 = 250 
    enc_dim2 = 50 
    enc_dim3 = k

    inp1 = Input(shape=(g,))

    enc_1 = Dense(enc_dim1, activation='relu')(inp1)
    enc_2 = Dense(enc_dim2, activation='relu')(enc_1)
    enc_3 = Dense(enc_dim3, activation='relu')(enc_2)
    z = Dense(g, trainable=False)(enc_3)


    encoder1 = Model(inp1,enc_3)

    model = Model(inp1, z)
    model.layers[4].set_weights([M.T,np.zeros((g,))])

    model.compile(optimizer='adadelta', loss='mean_squared_error')
    model.fit(dat_T, dat_T,
                epochs=25,
                batch_size=100,
                shuffle=True,
                validation_data=(dat_T, dat_T))

    W = encoder1.predict(dat_T)
    return(W)


In [99]:
def OptimizeM(W,dat,k):
    (g,c) = dat.shape

    enc_dim1 = 100 
    enc_dim2 = 50 
    enc_dim3 = k

    inp1 = Input(shape=(c,))

    enc_1 = Dense(enc_dim1, activation='relu')(inp1)
    enc_2 = Dense(enc_dim2, activation='relu')(enc_1)
    enc_3 = Dense(enc_dim3, activation='relu')(enc_2)
    z = Dense(c, trainable=False)(enc_3)


    encoder1 = Model(inp1,enc_3)

    model = Model(inp1, z)
    model.layers[4].set_weights([W,np.zeros((c,))])

    model.compile(optimizer='adadelta', loss='mean_squared_error')
    model.fit(dat, dat,
                epochs=25,
                batch_size=100,
                shuffle=True,
                validation_data=(dat, dat))

    M = encoder1.predict(dat)
    return(M)

In [109]:
def OuterLayer(dat,k,reps_unc = 2, reps_net = 2):
    print('Optimizing UNCURL')
    M_init, W_init, _ = run_state_estimation(dat, clusters=k, dist='Gaussian', reps = reps_unc)
    print('Optimizing UNCURL-net')
    
    M = M_init + 0.0 
    W = W_init + 0.0 
    
    for i in range(reps_net):
        
        print("Optimizing W iter" + str(i))
        W = OptimizeW(M,dat.T,k)
        W = W.T
        print("Optimizing M iter" + str(i))
        M = OptimizeM(W,dat,k)
        
        
    return((M_init,W_init,M,W))
        
    

In [110]:
#run UNCURL for a few iterations
k = 10
(M_init,W_init,M,W) = OuterLayer(dat,k)

Optimizing UNCURL
Optimizing UNCURL-net
Optimizing W iter0
Train on 400 samples, validate on 400 samples
Epoch 1/25
400/400 [==============================] - 83s 208ms/step - loss: 0.0246 - val_loss: 0.0169
Epoch 2/25
400/400 [==============================] - 0s 205us/step - loss: 0.0172 - val_loss: 0.0166
Epoch 3/25
400/400 [==============================] - 0s 199us/step - loss: 0.0163 - val_loss: 0.0140
Epoch 4/25
400/400 [==============================] - 0s 249us/step - loss: 0.0133 - val_loss: 0.0127
Epoch 5/25
400/400 [==============================] - 0s 189us/step - loss: 0.0125 - val_loss: 0.0123
Epoch 6/25
400/400 [==============================] - 0s 199us/step - loss: 0.0122 - val_loss: 0.0121
Epoch 7/25
400/400 [==============================] - 0s 209us/step - loss: 0.0121 - val_loss: 0.0120
Epoch 8/25
400/400 [==============================] - 0s 196us/step - loss: 0.0120 - val_loss: 0.0119
Epoch 9/25
400/400 [==============================] - 0s 202us/step - loss: 0.

In [123]:
#Compare relative accuracy of approaches on clustering 
c_unc = W_init.argmax(0)
c_net = W.argmax(0)
print('Basic UNCURL Purity ='+str(uncurl.evaluation.purity(mat['labels'][0],c_unc)))
print('Deep UNCURL Purity ='+str(uncurl.evaluation.purity(mat['labels'][0],c_net)))

Basic UNCURL Purity =0.7525
Deep UNCURL Purity =0.97
